## Кросс-валидация

Будем решать задачу классификации на примере известного датасета о цветках ирисов.


В этом датасете каждый объект (цветок) имеет четыре числовые характеристики: 
*    длина и ширина лепестка;
*    длина и ширина чашелистика.

Сначала для оценки качества модели используем отложенную выборку.

Затем применим только что изученную нами технику и будем оценивать качество модели при помощи кросс-валидации.

Затем сделаем выводы — посмотрим, какую информацию даст нам кросс-валидация относительно качества модели и её степени подгонки под данные.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.datasets import load_iris

data = load_iris(as_frame=True)

X = data.data[['sepal length (cm)', 'sepal width (cm)']]
y = data.target

Для простоты переведём задачу к бинарной классификации.

In [ ]:
y[y!=1] = -1

In [ ]:
X.head()

,sepal length (cm),sepal width (cm)
0,5.1,3.5
1,4.9,3.0
2,4.7,3.2
3,4.6,3.1
4,5.0,3.6


Разобьём данные на тренировочную и тестовую части. Затем обучим модель на тренировочных данных и оценим её качество на train и test.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)

y_train_pred = tree.predict(X_train)
y_test_pred = tree.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_train, y_train_pred), accuracy_score(y_test, y_test_pred)

(0.9583333333333334, 0.5666666666666667)

Заново разобьём данные на train и test и повторим процесс обучения и оценки качества модели.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
y_train_pred = tree.predict(X_train)
y_test_pred = tree.predict(X_test)

accuracy_score(y_train, y_train_pred), accuracy_score(y_test, y_test_pred)

(0.9166666666666666, 0.7333333333333333)

Получили другие значения метрик! То есть значения метрик зависят от разбиения данных на train и test.

Теперь используем кросс-валидацию k-fold для оценки качества.

В sklearn есть два способа её осуществить:
*  cross_val_score
*  cross_validate

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(DecisionTreeClassifier(), X, y, cv=3, scoring='accuracy')

In [ ]:
scores

array([0.78, 0.7 , 0.58])

Мы сделали кросс-валидацию с тремя фолдами и на каждом фолде получили различные значения accuracy. Визуально разброс значений довольно большой!

Посчитаем среднее значение и стандартное отклонение.

In [ ]:
scores.mean()

0.6866666666666666

In [ ]:
scores.std()

0.08219218670625304

Теперь по среднему значению accuracy на кросс-валидации мы получили более корректную оценку метрики качества.

Кроме того, значение стандартного отклонения говорит о том, что метрика довольно сильно зависит от разбиения данных. То есть может иметь место переобучение.

Второй способ — это cross_validate. Он отличается от cross_val_score тем, что:

*    позволяет измерять несколько метрик одновременно;
*    возвращает словарь, содержащий время обучения, время применения, значения метрик на тестовых данных (и опционально — значения метрик на тренировочных данных).

In [ ]:
from sklearn.model_selection import cross_validate

res = cross_validate(DecisionTreeClassifier(), X, y, cv=3, scoring=['accuracy','roc_auc'])

In [ ]:
res

{'fit_time': array([0.00589919, 0.00215006, 0.00199604]),
 'score_time': array([0.00893593, 0.00371289, 0.00353289]),
 'test_accuracy': array([0.78, 0.7 , 0.58]),
 'test_roc_auc': array([0.71875   , 0.67290553, 0.64795009])}

Из полученной информации мы также можем извлечь среднее качество на тестовых данных и стандартное отклонение.

In [ ]:
res['test_accuracy'].mean(), res['test_accuracy'].std()

(0.6866666666666666, 0.08219218670625304)

Кроме того, получаем довольно много дополнительной диагностической информации, такой как:
*   время обучения модели на каждом запуске кросс-валидации;
*   время применения модели.

Это полезная информация. Оценивая время обучения и время применения модели, мы можем оценить временные затраты на обучение и на применение модели на всём датасете, а также при увеличении количества данных.